In [8]:
import pandas as pd 
from datetime import datetime, date, time, timedelta

 
df = pd.read_csv("crime.csv") 

In [9]:
df['DATE'], df['TIME'] = df['FIRST_OCCURRENCE_DATE'].str.split(' ', 1).str


df['DATE'] = pd.to_datetime(df['DATE'])
df['day_of_week'] = df['DATE'].dt.day_name()

In [10]:
df_violent = df[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LON', 'GEO_LAT']]
df_violent = df_violent.loc[(df_violent.OFFENSE_CATEGORY_ID == 'aggravated-assult')|(df_violent.OFFENSE_CATEGORY_ID == 'sexual-assult')|(df_violent.OFFENSE_CATEGORY_ID == 'robbery')|(df_violent.OFFENSE_CATEGORY_ID == 'murder')].copy()
df_violent_am =  df_violent.loc[df_violent.TIME.str[-2:] == "AM" ].copy()
df_violent_am['TIME'] = [x[:-3] for x in df_violent_am['TIME']]
df_violent_am['TIME'] =  pd.to_datetime(df_violent_am['TIME'], format='%H:%M:%S')

df_violent = df_violent[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LON','GEO_LAT']]
df_violent = df_violent.loc[(df_violent.OFFENSE_CATEGORY_ID == 'aggravated-assult')|(df_violent.OFFENSE_CATEGORY_ID == 'sexual-assult')|(df_violent.OFFENSE_CATEGORY_ID == 'robbery')|(df_violent.OFFENSE_CATEGORY_ID == 'murder')].copy()
df_violent_pm =  df_violent.loc[df_violent.TIME.str[-2:] == "PM" ].copy()
df_violent_pm['TIME'] = [x[:-3] for x in df_violent_pm['TIME']]
df_violent_pm['TIME'] =  pd.to_datetime(df_violent_pm['TIME'], format='%H:%M:%S')
df_violent_pm['TIME'] = df_violent_pm['TIME'] + timedelta(hours=12)

In [11]:
final_violent = pd.concat([df_violent_am, df_violent_pm], ignore_index=True, sort =False)

block1 = '09/19/18 04:00:00'
block2 = '09/19/18 08:00:00'
block3 = '09/19/18 12:00:00'
block4 = '09/19/18 16:00:00'
block5 = '09/19/18 20:00:00'
block6 = '09/19/18 23:59:00'
datetime_object1 = datetime.strptime(block1, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%m/%d/%y %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%m/%d/%y %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%m/%d/%y %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%m/%d/%y %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%m/%d/%y %H:%M:%S')


In [12]:
final_violent.loc[final_violent['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object1.time())&(final_violent['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object2.time())&(final_violent['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object3.time())&(final_violent['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object4.time())&(final_violent['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object5.time())&(final_violent['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6



final_violent['TIME'] = [d.time() for d in final_violent['TIME']]

In [13]:
violent_csv = final_violent[['day_of_week','NEIGHBORHOOD_ID', 'TIME']]
violent_plot_csv = final_violent[['day_of_week','NEIGHBORHOOD_ID', 'TIME','GEO_LON', 'GEO_LAT']]
violent_csv = violent_csv.sample(6000)
violent_csv.to_csv('Violent.csv')
violent_plot_csv.to_csv('Violent_Plot.csv')


!python apriori.py -f 'Violent.csv' -s 0.003

item: ('five-points', '23:59:00', 'Friday') , 0.003
item: ('Sunday', 'union-station', '04:00:00') , 0.003
item: ('five-points', '23:59:00', 'Monday') , 0.003
item: ('union-station', '04:00:00', 'Saturday') , 0.004
